In [1]:
# Get the data for expeirment
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import geopy.distance as distance
import TsModel
import GprModel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RBF
import matplotlib.pyplot as plt
import pickle

import torch
import matplotlib.pyplot as plt
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
from keras import layers, models

device = torch.device("cuda:0")
print(f"Keras version is {keras.__version__}")
print(f"Num GPUs Available: {torch.cuda.device_count()}")
plt.rcParams.update({'font.size': 20})

Keras version is 3.1.1
Num GPUs Available: 1


In [74]:
# Train all models that will be used in the visualization project

In [2]:
def find_clostest_n_neighbours(target, unique_stations, number_of_neighbours):
    station_with_locations = unique_stations.copy()

    distances = station_with_locations.apply(
        lambda row: distance.distance(
            [row['latitude'], row['longitude']], [target[0], target[1]]).km,
        axis=1
    )
    station_with_locations['distance'] = distances

    station_in_range = station_with_locations.loc[(station_with_locations['distance'] >= 0)
                                                  & (station_with_locations['distance'] <= 99999)]

    station_to_use = station_in_range.nsmallest(number_of_neighbours, 'distance')
    
    return station_to_use


def extract_data(features, target_coor, features_to_use=None, target_features_to_use=None):
    if features_to_use is None:
        features_to_use = ['latitude', 'longitude', 'temp', 'wind_direction', 'wind_speed']
    if target_features_to_use is None:
        target_features_to_use = ['wind_speed']

    distances = features.apply(
        lambda row: distance.distance(
            [row['latitude'], row['longitude']], [target_coor[0], target_coor[1]]).km,
        axis=1
    )

    processed_features = features.loc[:,features_to_use].copy()
    processed_features['distance'] = distances

    processed_features = processed_features.to_numpy()

    return processed_features


# Note some station will have less data, so the smallest date range is used to mach all stations
def extract_data_match_date_range(features, target_coor, neighbour_station_names):
    processed_features = []

    for name in neighbour_station_names:
        selected_station_data = features.loc[features['name'] == name]
        extracted_features = extract_data(selected_station_data, target_coor)
        if len(processed_features) == 0:
            processed_features = extracted_features
        else:
            processed_features = np.concatenate((processed_features, extracted_features), axis=1)

    return processed_features    

# Given the target station name, find the nearest neighbours within the distance
def generate_data(raw_data, number_of_neighbours):
    output_features = []
    
    # All stations in the dataset
    stations_to_test = [
        'LETHBRIDGE CDA',                 
        'EDMONTON STONY PLAIN CS',        
        'CORONATION CLIMATE',             
        'STRATHMORE AGDM',                
        'LLOYDMINSTER',                  
        'MEDICINE HAT RCS',             
        'MILK RIVER',                     
        'CAMROSE',                     
        'BROOKS',                    
        'CLARESHOLM',           
        'ONEFOUR CDA',            
        'VEGREVILLE',             
        'ROCKY MTN HOUSE (AUT)',         
        'LACOMBE CDA 2',              
        'BANFF CS',                     
        'DRUMHELLER EAST',         
        'MEDICINE HAT',                 
        'LETHBRIDGE',                 
        'EDMONTON INTL A',         
        'CALGARY INTL A'
    ]

    for station_name in stations_to_test:
        target = raw_data.loc[raw_data['name'] == station_name]
        target_latitude = target.iloc[0]['latitude']
        target_longitude = target.iloc[0]['longitude']

        # select all unique names and coordinates
        unique_stations_names = raw_data.groupby('name').head(1)
        unique_stations_names = unique_stations_names.loc[unique_stations_names['name'] != station_name]
        neighbour_stations = find_clostest_n_neighbours([target_latitude, target_longitude], unique_stations_names, number_of_neighbours)

        # find k nearest neighbours
        neighbour_station_names = neighbour_stations['name']

        # filter the data, return
        features = raw_data[raw_data['name'].isin(neighbour_station_names)]
        features = extract_data_match_date_range(features, [target_latitude, target_longitude], neighbour_station_names)
        output_features.extend(features)

    return features, target.loc[:,['wind_speed']].to_numpy()


In [2]:
train_df = pd.read_csv('data/processed_ab_wind_train.txt')

C:\Users\PPL\AppData\Local\Temp\ipykernel_22432\3303841789.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('data/processed_ab_wind_train.txt')


In [33]:
train_x, train_y = generate_data(train_df, 9)

In [34]:
test_df = pd.read_csv('data/processed_ab_wind_test.txt')
test_x, test_y = generate_data(test_df, 9)

C:\Users\PPL\AppData\Local\Temp\ipykernel_23364\3486938129.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('data/processed_ab_wind_test.txt')


In [35]:
temp_x = pd.DataFrame(train_x)
temp_x.to_csv(f"Data\exp\9x_train.csv")
temp_y = pd.DataFrame(train_y)
temp_y.to_csv(f"Data\exp\9y_train.csv")

temp_x = pd.DataFrame(test_x)
temp_x.to_csv(f"Data\exp\9x_test.csv")
temp_y = pd.DataFrame(test_y)
temp_y.to_csv(f"Data\exp\9y_test.csv")

In [36]:
def build_neural_network_model():
    model = models.Sequential()
    model.add(layers.Dense(30, activation=keras.activations.tanh))    
    model.add(layers.Dense(1, activation=keras.activations.relu))
    return model

In [4]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

train_x, train_y = shuffle(train_x, train_y)

In [38]:
neural_network_model = build_neural_network_model()


neural_network_model.compile(
    optimizer='adam',
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.RootMeanSquaredError(), keras.metrics.MeanAbsolutePercentageError()]
)

history = neural_network_model.fit(
    train_x, 
    train_y, 
    epochs=50, 
    shuffle=True
)

Epoch 1/50
835/835 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 230.8481 - mean_absolute_percentage_error: 81.5564 - root_mean_squared_error: 15.1156
Epoch 2/50
835/835 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 52.0095 - mean_absolute_percentage_error: 54.2399 - root_mean_squared_error: 7.2091
Epoch 3/50
835/835 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 47.6094 - mean_absolute_percentage_error: 53.8312 - root_mean_squared_error: 6.8999
Epoch 4/50
835/835 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 45.6105 - mean_absolute_percentage_error: 53.0362 - root_mean_squared_error: 6.7532
Epoch 5/50
835/835 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 45.9449 - mean_absolute_percentage_error: 53.4937 - root_mean_squared_error: 6.7780
Epoch 6/50
835/835 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 44.5324 - mean_absolute_percentage_error: 52.1871 - root_mean_squared_error: 6.6730
Epoch 7/50
835/835 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 43.8564 - mean_absolute_percentage_error: 52.4177 - root_mean_squared_erro

In [39]:
#neural_network_model.save('Visualization_proj_9stations.keras')

In [3]:
train_x = pd.read_csv(f"Data\exp\9x_train.csv")
train_x = train_x.drop(columns=['Unnamed: 0']).to_numpy()
train_y = pd.read_csv(f"Data\exp\9y_train.csv")
train_y = train_y.to_numpy()[:,1].reshape(-1,1)
test_x = pd.read_csv(f"Data\exp\9x_test.csv")
test_x = test_x.drop(columns=['Unnamed: 0']).to_numpy()
test_y = pd.read_csv(f"Data\exp\9y_test.csv")
test_y = test_y.to_numpy()[:,1].reshape(-1,1)

In [39]:
indices = np.random.choice(train_x.shape[0], 2000, replace=False)            

# Martern kernel
kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
gpr_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gpr_model.fit(train_x[indices,:], train_y[indices,:])

GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=1, nu=1.5),
                         n_restarts_optimizer=9)

In [40]:
predicted_means, predicted_std = gpr_model.predict(test_x, return_std=True)

In [42]:
gpr_model.score(test_x, test_y)

0.41631521833213103

In [50]:
# Save the model
with open('gp_martern_model.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(gpr_model, f, pickle.HIGHEST_PROTOCOL)

In [5]:
# Train the TS model
# Optimal m= 1.2, c=30
ts_model = TsModel.TsModel(number_of_rules=30, fuzzification_coefficient=1.2)
ts_model.fit(train_x,train_y)
ts_score = r2_score(test_y, ts_model.predict(test_x))
print(ts_score)

FCM training RMSE: 6.079000613380315
0.5170380406914143


In [6]:
# # Save the model
with open('ts_model.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(ts_model, f, pickle.HIGHEST_PROTOCOL)

In [18]:
# Now use the prototype and conclusion of the TS model to train the GPR model
# Get the respective output of the TS model for the inputs that coincides with the prototypes
gp_ts_output = ts_model.predict(ts_model.cen + 1e-8)

In [27]:
kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
gpr_model_ts_martern = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gpr_model_ts_martern.fit(ts_model.cen, gp_ts_output)

GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=1, nu=1.5),
                         n_restarts_optimizer=9)

In [28]:
gpr_model_ts_martern.score(test_x, test_y)

0.4268837173170462

In [29]:
# Save the model
with open('gp_martern_ts_model.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(gpr_model_ts_martern, f, pickle.HIGHEST_PROTOCOL)